In [1]:
#import the required libary
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Reviews.csv', low_memory=False)#read the csv file with dataset
print(df.columns)#shows the column in dataset
print(df.shape)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
(568454, 10)


In [3]:
df['Score'] = pd.to_numeric(df['Score'], errors='coerce')
print(df.info())#show the data information whether int otr object or float


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB
None


In [4]:
#making the new df 
rating_df=pd.read_csv('Reviews.csv',usecols=['Id', 'ProductId', 'UserId', 'ProfileName','Score', 'Time', 'Summary'],
dtype={'Id':'int64','ProductId':'object','UserID':'object','ProfileName':'object','Score':'int64','Time':'int64','Summanry':'object'})

In [5]:
rating_df.head()

,Id,ProductId,UserId,ProfileName,Score,Time,Summary
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,5,1303862400,Good Quality Dog Food
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,1,1346976000,Not as Advertised
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",4,1219017600,"""Delight"" says it all"
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,2,1307923200,Cough Medicine
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",5,1350777600,Great taffy


In [6]:
rating_df2=rating_df[:5000]
#using subset 

In [7]:
#getting the total count of product
product_rating = rating_df2.dropna(axis = 0, subset = ['ProductId'])
product_ratingCount = (product_rating.
     groupby(by = ['ProductId'])['Score'].
     count().
     reset_index().
     rename(columns = {'Score': 'totalScoreCount'})
     [['ProductId', 'totalScoreCount']]
    )
product_ratingCount.head()

,ProductId,totalScoreCount
0,B00002NCJC,2
1,B00002Z754,2
2,B000084DVR,2
3,B000084E1U,1
4,B0000CGFV4,1


In [8]:
#making new df
rating_with_totalRatingCount = product_rating.merge(product_ratingCount, left_on = 'ProductId', right_on = 'ProductId', how = 'left')
rating_with_totalRatingCount.head()

,Id,ProductId,UserId,ProfileName,Score,Time,Summary,totalScoreCount
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,5,1303862400,Good Quality Dog Food,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,1,1346976000,Not as Advertised,1
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",4,1219017600,"""Delight"" says it all",1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,2,1307923200,Cough Medicine,1
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",5,1350777600,Great taffy,4


In [9]:
#change the formate
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(product_ratingCount['totalScoreCount'].describe())

count   731.000
mean      6.840
std      22.688
min       1.000
25%       1.000
50%       2.000
75%       5.000
max     324.000
Name: totalScoreCount, dtype: float64


In [10]:
#list with more thant 50 score
popularity_threshold = 50
rating_popular_product= rating_with_totalRatingCount.query('totalScoreCount >= @popularity_threshold')
rating_popular_product.head()

,Id,ProductId,UserId,ProfileName,Score,Time,Summary,totalScoreCount
422,423,B000G6RYNE,A2NO1TXXS9T0EE,BruteNStones,4,1335398400,Hard,217
423,424,B000G6RYNE,A1IRN1M05TPOVT,"Sharon M. Helfand ""Scrapper""",5,1334620800,Kettle potato chips: Fully loaded baked potato,217
424,425,B000G6RYNE,ARONI7W0W677H,Peter N,5,1334275200,"Fresh, Lightly-Spiced, Crunchy Kettle Chips. G...",217
425,426,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,4,1334016000,glad to find them in 1 oz size,217
426,427,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,4,1334016000,"pretty good, could be better",217


In [11]:
rating_popular_product.shape


(1772, 8)

In [12]:
rating_popular_product


,Id,ProductId,UserId,ProfileName,Score,Time,Summary,totalScoreCount
422,423,B000G6RYNE,A2NO1TXXS9T0EE,BruteNStones,4,1335398400,Hard,217
423,424,B000G6RYNE,A1IRN1M05TPOVT,"Sharon M. Helfand ""Scrapper""",5,1334620800,Kettle potato chips: Fully loaded baked potato,217
424,425,B000G6RYNE,ARONI7W0W677H,Peter N,5,1334275200,"Fresh, Lightly-Spiced, Crunchy Kettle Chips. G...",217
425,426,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,4,1334016000,glad to find them in 1 oz size,217
426,427,B000G6RYNE,A1Y3XPZK9ZADFW,albinocrow,4,1334016000,"pretty good, could be better",217
...,...,...,...,...,...,...,...,...
4856,4857,B00139TT72,A2TIC47QPFZ10W,Susan Eagle,5,1349654400,My dog loves it,159
4857,4858,B00139TT72,A3FJ0YTQ4TR4QL,"Rebecca K-B ""RAK-B""",5,1349654400,Quality pet food,159
4858,4859,B00139TT72,A2I334K2Y6VN2E,Bonita,5,1349568000,Excellant Product.,159
4859,4860,B00139TT72,A2666ZSQOFE0D8,noopiemom,4,1349568000,My dog's liked it but wanted some variety,159


In [13]:
#making pivot table
table=rating_popular_product.pivot_table(index='ProductId',columns='UserId',values='Score')

In [14]:
table

UserId,A103U3KR4L2ZXT,A107MO1RZUQ8V,A107SVKYGPGBPP,A108GQ9A91JIP4,A10APXLM7P5P0G,A10E57QZ4GJ32F,A10RJEQN64ATXU,A10VCCF3QS34YL,A10ZBR6O8S8OCY,A10ZN08SMLZL1Y,...,AZ6DJLNAUGTUG,AZ81DUK0S2SHD,AZ9N3928BGQ0E,AZE0XC66FVSDE,AZG9NISRWJB6F,AZI5EI9VX00Y,AZS05OYE0XGNF,AZVQMPXZEHQGZ,AZVR7NPPEDMMN,AZYJE40XW6MFG
ProductId,,,,,,,,,,,,,,,,,,,,,
B000G6MBX2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B000G6RYNE,NaN,NaN,NaN,NaN,NaN,NaN,5.000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00139TT72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0016FY6H6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B001E5E29A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B001EW5YQS,NaN,NaN,NaN,NaN,5.000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000,NaN,NaN
B001RVFDOO,NaN,5.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.000,NaN,NaN,NaN,NaN,5.000,4.000,NaN,3.000,NaN
B002TMV3E4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000
B003NDA970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#Collaborative filtering with KNN (K- nearest neighbor)
from scipy.sparse import csr_matrix
from  sklearn.neighbors import NearestNeighbors

In [16]:
#fill the null with 0
table=table.fillna(0)

In [17]:
table


UserId,A103U3KR4L2ZXT,A107MO1RZUQ8V,A107SVKYGPGBPP,A108GQ9A91JIP4,A10APXLM7P5P0G,A10E57QZ4GJ32F,A10RJEQN64ATXU,A10VCCF3QS34YL,A10ZBR6O8S8OCY,A10ZN08SMLZL1Y,...,AZ6DJLNAUGTUG,AZ81DUK0S2SHD,AZ9N3928BGQ0E,AZE0XC66FVSDE,AZG9NISRWJB6F,AZI5EI9VX00Y,AZS05OYE0XGNF,AZVQMPXZEHQGZ,AZVR7NPPEDMMN,AZYJE40XW6MFG
ProductId,,,,,,,,,,,,,,,,,,,,,
B000G6MBX2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B000G6RYNE,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B00139TT72,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B0016FY6H6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B001E5E29A,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
B001EW5YQS,0.000,0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,0.000,0.000
B001RVFDOO,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,5.000,4.000,0.000,3.000,0.000
B002TMV3E4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000
B003NDA970,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [18]:
#arranging into matrix
matrix=csr_matrix(table.values)

In [19]:
model_knn = NearestNeighbors(metric= 'cosine', algorithm= 'brute')
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
#giving index for recommended
user_query_index = 3
user_query_index

3

In [21]:
table.index[user_query_index]

'B0016FY6H6'

In [22]:
table.iloc[user_query_index, :]

UserId
A103U3KR4L2ZXT   0.000
A107MO1RZUQ8V    0.000
A107SVKYGPGBPP   0.000
A108GQ9A91JIP4   0.000
A10APXLM7P5P0G   0.000
                  ... 
AZI5EI9VX00Y     0.000
AZS05OYE0XGNF    0.000
AZVQMPXZEHQGZ    0.000
AZVR7NPPEDMMN    0.000
AZYJE40XW6MFG    0.000
Name: B0016FY6H6, Length: 1723, dtype: float64

In [36]:
distances, indices = model_knn.kneighbors(table.iloc[user_query_index, :].values.reshape(1, -1), n_neighbors = 6)

In [37]:
for i in range(0, len(distances.flatten())):
    # the below line will be printed anyway
    # we always start i = 0
    # just getting the variable i ready to print for which product we are
    # generating the recommendation for:
    if i == 0:
        print('Recommendation for {0}:\n'.format(table.index[user_query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, table.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendation for B0016FY6H6:

1: B002TMV3E4, with distance of 0.9813865330508962:
2: B004391DK0, with distance of 1.0:
3: B005K4Q1VI, with distance of 1.0:
4: B001EW5YQS, with distance of 1.0:
5: B003NDA970, with distance of 1.0:


In [25]:
df3=df[:1000]
df3

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
995,996,B006F2NYI2,A1D3F6UI1RTXO0,Swopes,1,1,5,1331856000,Hot & Flavorful,BLACK MARKET HOT SAUCE IS WONDERFUL.... My hus...
996,997,B006F2NYI2,AF50D40Y85TV3,Mike A.,1,1,5,1328140800,Great Hot Sauce and people who run it!,"Man what can i say, this salsa is the bomb!! i..."
997,998,B006F2NYI2,A3G313KLWDG3PW,kefka82,1,1,5,1324252800,this sauce is the shiznit,this sauce is so good with just about anything...
998,999,B006F2NYI2,A3NIDDT7E7JIFW,V. B. Brookshaw,1,2,1,1336089600,Not Hot,Not hot at all. Like the other low star review...


In [43]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,3),min_df=3,analyzer='word')

#Replace NaN with an empty string
df3['Text'] = df3['Text'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df3['Text'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(1000, 3063)

In [44]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [45]:
print(cosine_sim.shape)
print(cosine_sim[1])

(1000, 1000)
[0.07512916 1.         0.         0.         0.         0.
 0.         0.         0.         0.05080291 0.         0.02774269
 0.         0.         0.         0.         0.         0.
 0.01874353 0.         0.         0.02248462 0.04189538 0.
 0.         0.05421439 0.         0.         0.         0.
 0.         0.03991902 0.02311209 0.02184616 0.         0.
 0.02781084 0.         0.         0.01672325 0.03209726 0.01386092
 0.03640466 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.30077511 0.
 0.         0.         0.10514919 0.         0.04043877 0.
 0.         0.         0.10956025 0.08262516 0.08974729 0.
 0.         0.         0.         0.         0.         0.
 0.03463996 0.         0.         0.         0.03695499 0.
 0.         0.         0.         0.         0.04152054 0.
 0.         0.         0.         0.         0.         0.03629206
 0.         0.         0.         0.         0.         0.04100088
 0.01010793

In [46]:
#Construct a reverse map of indices and product ID
indices = pd.Series(df3.index, index=df3['ProductId']).drop_duplicates()

In [47]:
def get_recommendations(ProductId, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the ProductID
    idx = indices[ProductId]

    # Get the pairwsie similarity scores of all movies with that product
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar product
    sim_scores = sim_scores[1:11]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar product
    return df3['ProductId'].iloc[product_indices]

In [48]:
Output=get_recommendations('B00813GRG4')
Output

52     B000G6RPMY
384    B004AVYUOW
584    B000G6RYNE
366    B00437JI8Q
389    B001ESOQA2
646    B002HQH04O
409    B001IZM8A6
505    B000G6RYNE
963    B000YT5DBS
342    B000HKYP9A
Name: ProductId, dtype: object